# Bleu scores on backtranslation of summary 

In [1]:
# save GEMINI_API_KEY in .env file
# call `huggingface-cli login` to login to huggingface
import os
import dotenv
import json

dotenv.load_dotenv()

True

In [2]:
source_languages = [
    "english",
    "french",
    "german",
    "hindi",
    "portuguese",
    "spanish",
]
target_languages = {
    "english": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french",],
    "french": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish",],
    "german": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french",],
    "hindi": ["bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french",],
    "portuguese": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french",],
    "spanish": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "french",],
}
datasets = {}
for s in source_languages:
    with open(f"./codesearchnet_summary_{s}.json", "r") as f:
        datasets[s] = json.load(f)


datasets["english"][0]

{'code': 'def addidsuffix(self, idsuffix, recursive = True):\n        """Appends a suffix to this element\'s ID, and optionally to all child IDs as well. There is sually no need to call this directly, invoked implicitly by :meth:`copy`"""\n        if self.id: self.id += idsuffix\n        if recursive:\n            for e in self:\n                try:\n                    e.addidsuffix(idsuffix, recursive)\n                except Exception:\n                    pass',
 'docstring': "Appends a suffix to this element's ID, and optionally to all child IDs as well. There is sually no need to call this directly, invoked implicitly by :meth:`copy`",
 'summary': 'Sure! Let’s break down what this function does in simple terms:\n\n1. **Purpose**: The function `addidsuffix` adds a suffix (a piece of text) to the `id` of an element. It can also optionally add the same suffix to the `id` of all its child elements.\n\n2. **Parameters**:\n   - `idsuffix`: This is the text you want to append to the `i

In [3]:
from google import genai
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [11]:
source = "english"

prompt_tmpl = """
Translate the following code from {source} to {target}
Don't include any other text than the translation.
Text: {text}
Translation:
"""

for sample in datasets[source]:
    target_lang = target_languages[source][0]
    summary = sample["summary"]
    # translate summary source -> target
    summary_target = client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt_tmpl.format(source=source, target=target_lang, text=summary)
    ).text
    # translate summary target -> source
    summary_back = client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt_tmpl.format(source=target_lang, target=source, text=summary_target)
    ).text
    # save results
    sample[f"summary_{source}_{target_lang}"] = summary_target
    sample[f"summary_{target_lang}_{source}"] = summary_back


In [12]:
# calculate blue and meteor scores
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

def calculate_bleu(reference, candidate):
    """Calculate BLEU score between reference and candidate strings"""
    reference_tokens = [word_tokenize(reference.lower())]
    candidate_tokens = word_tokenize(candidate.lower())
    return sentence_bleu(reference_tokens, candidate_tokens)

# Calculate BLEU score between original and translated text
for sample in datasets[source]:
    target_lang = target_languages[source][0]
    text1 = sample[f"summary"]
    text2 = sample[f"summary_{target_lang}_{source}"]
    bleu_score = calculate_bleu(text1, text2)
    sample[f"bleu_summary_{target_lang}"] = bleu_score


In [15]:
import pandas as pd

columns = ["sample_id", "source", "target", "bleu"]
data = []
for idx, sample in enumerate(datasets[source]):
    target_lang = target_languages[source][0]
    data.append(
        [
            idx,
            source,
            target_lang,
            sample[f"bleu_summary_{target_lang}"],
        ]
    )
df_bleu = pd.DataFrame(data, columns=columns)


In [16]:
df_bleu.to_csv("bleu_summary.csv", index=False)

In [17]:
df_bleu.head()

,sample_id,source,target,bleu
0,0,english,hindi,0.884151
1,1,english,hindi,0.814762
2,2,english,hindi,0.889584
3,3,english,hindi,0.896659
4,4,english,hindi,0.763425


In [18]:
df_bleu.describe()

,sample_id,bleu
count,100.000000,100.000000
mean,49.500000,0.843061
std,29.011492,0.045371
min,0.000000,0.734239
25%,24.750000,0.809586
50%,49.500000,0.844386
75%,74.250000,0.875473
max,99.000000,0.946186
